<a href="https://colab.research.google.com/github/bperz/big_data/blob/master/lab5/BDA_l5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/My\ Drive
%cd BDA\ resources

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME
# findspark.init("/gdrive/My\ Drive/BDA\ resources")
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

In [0]:
import re, nltk
nltk.download('stopwords')

# Task 1

In [0]:
graph = [
[1, [3,4,5]],
[2, [1,3]],
[3, [4,5]],
[4, [1,2]],
[5, [4,5]]
]


In [0]:
graph

In [0]:
data = sc.parallelize(graph)

In [0]:
data.take(2)

In [0]:
mapped = data.flatMap(lambda x: [[i, [x[0]]] for i in x[1]])

In [0]:
reduced = mapped.reduceByKey(lambda acc, n: acc+n).sortByKey().map(lambda x: list(x))

In [0]:
reduced.take(5)

# Task 2

In [0]:
def tuple_sum(tuple1, tuple2):
  return (tuple1[0]+tuple2[0], tuple1[1]+tuple2[1])

In [0]:
graph = []
with open('web-Stanford.txt') as file:
  for line in file:
    line_list = line.split()
    graph.append((int(line_list[0]), int(line_list[1])))

graph[:10]

test graph

In [0]:
# graph = [(3, 1),
#  (4, 1),
#  (5, 1),
#  (1, 2),
#  (3, 2),
#  (4, 3),
#  (5, 3),
#  (1, 4),
#  (2, 4),
#  (4, 5),
#  (5, 5)]


# graph = [(3, 1),
#  (4, 1),
#  (5, 1),
#  (2, 1),
#  (1, 1)]

In [0]:
data = sc.parallelize(graph)

In [0]:
mapped = data.flatMap(lambda x: [(x[0], (0, 1)), (x[1], (1, 0))])

In [0]:
reduced = mapped.reduceByKey(tuple_sum).sortByKey()

In [0]:
reduced.take(20)

count

In [0]:
seqOp = (lambda local_result, list_element: [local_result[0]+1, 
                                             local_result[1]+list_element[1][0], 
                                             local_result[2]+list_element[1][1]])

combOp = (lambda local_result_1, local_result_2: [local_result_1[0] + local_result_2[0],
                                                  local_result_1[1] + local_result_2[1],
                                                  local_result_1[2] + local_result_2[2]])

In [0]:
agg = reduced.aggregate([0, 0, 0], seqOp, combOp)

In [0]:
agg

avg

In [0]:
data = sc.parallelize(graph)

In [0]:
mapped = data.map(lambda x: (x[0], 1))

In [0]:
reduced = mapped.reduceByKey(lambda acc, n: acc+n)
reduced.take(5)

In [0]:
seqOp = (lambda local_result, list_element: [local_result[0]+1, 
                                             local_result[1]*(local_result[0]/(local_result[0]+1))+list_element[1][0]/(local_result[0]+1)])

combOp = (lambda local_result_1, local_result_2: [local_result_1[0] + local_result_2[0],
                                                  local_result_1[1]*(local_result_1[0]/(local_result_1[0] + local_result_2[0])) + local_result_2[1]*(local_result_2[0]/(local_result_1[0] + local_result_2[0]))])
          
agg = reduced.aggregate([0, 0], seqOp, combOp)

In [0]:
agg

[281903, 8.203165627893167]

Task 3

you can try ignoring leafs by giving them -1 while calculating mean (0 is also an information)


In [0]:
import time
start = time.time()

In [0]:
# def ordered_tuples(x):
#   if x[0] < x[1]:
#     return (x[0], x[1])
#   elif x[0] > x[1]:
#     return (x[1], x[0])
#   else:
#     return ()

def ordered_tuples(x):
  if x[0] != x[1]:
    return [(x[0], [x[1]]), (x[1], [x[0]])]
  else:
    return ()

In [0]:
graph = [(3, 1),
 (4, 1),
 (5, 1),
 (1, 2),
 (3, 2),
 (4, 3),
 (5, 3),
 (1, 4),
 (2, 4),
 (4, 5),
 (5, 5)]

In [0]:
data = sc.parallelize(graph)

In [0]:
undirectedG = data.flatMap(ordered_tuples).filter(lambda x: x!=())

In [0]:
neighbours = undirectedG.reduceByKey(lambda agg, x: agg+x)
neighbours = neighbours.map(lambda x: (x[0], set(x[1])))

nghb_dict = neighbours.collectAsMap()

In [0]:
vertices_properties = neighbours.map(lambda x: (x[0], 
                          sum([len(nghb_dict[x[0]].intersection(nghb_dict[nghb]))/(len(x[1])*(len(x[1])-1)) for i, nghb in enumerate(x[1]) if len(nghb_dict[nghb]) > 1 and len(nghb_dict[x[0]]) > 1]),
                          len(x[1])))

In [0]:
vertices_properties.take(5)

In [0]:
seqOp = (lambda local_result, list_element: [local_result[0]+1, 
                                             local_result[1]*(local_result[0]/(local_result[0]+1))+list_element[1]/(local_result[0]+1)])

combOp = (lambda local_result_1, local_result_2: [local_result_1[0] + local_result_2[0],
                                                  local_result_1[1]*(local_result_1[0]/(local_result_1[0] + local_result_2[0])) + local_result_2[1]*(local_result_2[0]/(local_result_1[0] + local_result_2[0]))])
          
agg = vertices_properties.aggregate([0, 0], seqOp, combOp)

In [0]:
agg

[281903, 0.5976304608027434]

In [0]:
time.time() - start